In [1]:
import pandas as pd
import numpy as np
import random

In [ ]:
cutoffs = pd.read_csv('clgBranchCutoff.csv')
cutoffs = cutoffs.iloc[::2]
cutoffs = cutoffs.drop(columns = ['Unnamed: 0', 'GNT2S', 'LNT2S', 'ORPHAN', 'EWS', 'TFWS', 'ABCD', 'EFGH', 'Unnamed: 16', 'Unnamed: 17'])
cutoffs = cutoffs.iloc[:681]

In [ ]:
clgBranch = pd.read_csv('clgNameBranch.csv')
clg = clgBranch.iloc[::2]
branch = clgBranch.iloc[1::2]
clg = list(clg['clg'])
branch = list(branch['clg'])
cutoffs['ClgName'] = clg
cutoffs['Branch'] = branch
colnames = ['ClgName', 'Branch', 'GOPENS', 'GSCS', 'GSTS', 'GNT1S', 'GOBCS', 'LOPENS', 'LOBCS', 'PWDOPENS']
cutoffs = cutoffs.reindex(columns=colnames)
cutoffs
colname = ['GOPENS', 'GSCS', 'GSTS', 'GNT1S', 'GOBCS', 'LOPENS', 'LOBCS', 'PWDOPENS']
cutoffs = cutoffs.replace(r'\r.*', '', regex=True)
cutoffs = cutoffs.replace(r'.*- ', '', regex=True)
cutoffs

In [ ]:
cutoffs.to_csv("cutoffs.csv")

In [ ]:
def extract_score_rank(string):
    info = list(string.split())
    info[1] = info[1][1:-1]
    return float(info[0]),float(info[1])

def extract_name_town_city(sample):
    res = sample.split(',')
    name, town, city = '', '', ''
    if len(res) >= 1: name = res[0]
    if len(res) >= 2: town = res[1]
    if len(res) >= 3: city = res[2]
    return name, town, city


def extraction(file_names: list, extenstions: list):
    res = []
    for index, file_name in enumerate(file_names):
        table = pd.read_csv(file_name)

        for i in range(table.shape[0]):
            if str(table['Score'][i]) == "nan":
                table['ClgName'][i-1] = str(table['ClgName'][i-1])
                table['ClgName'][i-1] += ", " + str(table['ClgName'][i])
        table = table.replace(r'.*- ', '', regex=True)
        table = table.replace(r'.*CET', 'MHTCET', regex=True)
        table = table.replace(r'\n', '', regex=True)


        table = table.drop(columns=['ChoiceCode'])
        table = table.dropna(subset=['Score'])
        
        rank = 'Rank' + extensions[index]
        scores = 'Score' + extensions[index]
        table[rank], table[scores] = zip(*table['Score'].map(extract_score_rank))
        table['Name'], table['Town'], table['City'] = zip(*table['ClgName'].map(extract_name_town_city))
        
        table = table.replace(r'.*- ', '', regex=True)
        
        table = table.drop(columns=['Score', 'ClgName'])
        table = table.reindex(columns=[rank, scores, 'Name', 'Branch', 'Town', 'City', 'Exam'])
        res.append(table)
    return res

In [ ]:
file_names = ['jee19-20cap1.csv', 'jee19-20cap2.csv', 'jee20-21cap1.csv', 'jee20-21cap2.csv']
extensions = ['_19_R1', '_19_R2', '_20_R1', '_20_R2']
df = extraction(file_names, extensions)

In [ ]:
merge_on = ['Name', 'Branch']

res = df[0]
for dataframe in df[1:]:
    res = pd.merge(res, dataframe, left_on=merge_on, right_on=merge_on)

res.head()

In [ ]:
res.iloc[3, :]

In [ ]:
res.iloc[4, :]

In [ ]:
colnames = [ 'Name', 'Branch', 'Town', 'City', 'Exam', 'Rank_19_R1', 'Score_19_R1',
       'Rank_19_R2', 'Score_19_R2', 'Rank_20_R1', 'Score_20_R1', 'Rank_20_R2',
       'Score_20_R2']
res = res.reindex(columns=colnames)

In [ ]:
df = pd.read_csv('final.csv')
moreinfo = pd.read_csv('moreclgdata.csv')
moreinfo.drop(moreinfo.iloc[:, 15:],inplace=True, axis=1)
df = pd.merge(df, moreinfo, left_on='Name', right_on='Name')
df = df.drop(columns=['Courses', 'City_x'])

In [ ]:
df.info()

In [ ]:
df['Rating'] = df['Rating'].astype(float)
df['Rating'][:5]

In [ ]:
# def assign_ratings(sample, index):
#     if sample: return sample
    
#     rank = df['Rank_20_R1'][index]
#     if rank < 400: df['Rating'] = random.uniform(3.5, 4)
#     elif rank < 800: df['Rating'] = random.uniform(3, 3.5)
#     elif rank < 1200: df['Rating'] = random.uniform(2.5, 3)
#     elif rank < 5000: df['Rating'] = random.uniform(2, 2.5)
#     else: df['Rating'] = random.uniform(1, 2)
    
for i in range(df.shape[0]):
    rank = df['Rank_20_R1'][i]
    if rank < 700: df['Rating'][i] = random.uniform(3.5, 4)
    elif rank < 1100: df['Rating'][i] = random.uniform(3, 3.5)
    elif rank < 2000: df['Rating'][i] = random.uniform(2.5, 3)
    elif rank < 5000: df['Rating'][i] = random.uniform(2, 2.5)
    else: df['Rating'][i] = random.uniform(1, 2)
    
df = df.round({'Rating':2})

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.to_csv('database.csv')

In [ ]:
df[0].head()

In [ ]:
df[1].head()

In [ ]:
res.columns

In [ ]:
res.head()

In [ ]:
res.info()

In [ ]:
res.to_csv('final.csv')